# 多输入多输出通道

实现一下多输入通道互相关运算

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

验证互相关运算的输出

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

计算多个通道的输出的互相关函数

In [3]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

# 连接（concatenate）张量（tensors）沿一个新维度
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [4]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 1x1卷积

$k_h=k_w=1$：融合通道信息！！！

In [5]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

## zip的用法

In [6]:
list1 = [1, 2, 3]  
list2 = ['a', 'b', 'c']  
zipped = zip(list1, list2)  
  
# 在Python 3中，zip返回一个迭代器，我们需要将其转换为列表来查看其内容  
print(list(zipped))  
# 输出：[(1, 'a'), (2, 'b'), (3, 'c')]

[(1, 'a'), (2, 'b'), (3, 'c')]


In [7]:
list1 = [1, 2, 3]  
list2 = ['a', 'b']  
zipped = zip(list1, list2)  
print(list(zipped))  
# 输出：[(1, 'a'), (2, 'b')]  
# 注意，zip停止在最短的输入迭代完成的时候

[(1, 'a'), (2, 'b')]


In [8]:
zipped = [(1, 'a'), (2, 'b'), (3, 'c')]  
list1, list2 = zip(*zipped)  
  
# 由于zip返回的是元组的元组，我们再次使用zip(*)来解压它  
# 然后将解压出的迭代器转换为列表  
list1 = list(list1)  
list2 = list(list2)  
  
print(list1)  
# 输出：[1, 2, 3]  
print(list2)  
# 输出：['a', 'b', 'c']

[1, 2, 3]
['a', 'b', 'c']


In [9]:
list1 = [1, 2, 3]  
list2 = ['a', 'b', 'c']  
list3 = list(zip(list1, list2))  
  
# 使用*操作符将元组列表解压到单独的变量中  
a, b = zip(*list3)  
a = list(a)  
b = list(b)  
  
print(a)  
# 输出：[1, 2, 3]  
print(b)  
# 输出：['a', 'b', 'c']

[1, 2, 3]
['a', 'b', 'c']
